## Dilated Convolution

이미지 처리 이외에도 CNN은 시계열에 성공적으로 적용돼왔다. 예를 들어 WaveNet은 오디오 처리를 위해 개발된 CNN이다. 시계열은 흔히 장기 상관관계를 보인다. 게다가 종속 변수는 lag prediction 변수에 대해 비선형 의존성을 보인다. WaveNet 구조는 다음 형태의 비선형 $p-$자기회귀이다.

$$y_t = \sum_{i=1}^p \phi_i(x_{t-i}) + \epsilon_t$$

여기서 계수 함수 $\phi_i, i \in \{1, \dots, p\}$는 데이터 의존적이고 합성곱 신경망을 통해 최적화된다. 신경망이 이들 장기 비선형 의존성을 학습할 수 있게 Borovtkh 등(2017)은 dilated convolution의 적층 구조를 사용한다. dilated convolution은 효과적으로 신경망이 보통 합성곱보다 더 희소한 스케일로 작동하게 한다. 이는 pooling 또는 stride 합성곱과 유사하지만 여기서 출력은 입력과 동일한 크기를 갖는다.(van den Oord et al. 2016) dilated convolution에서 필터는 입력 벡터의 모든 $d$번 째 원소에 적용돼 모델이 효율적으로 멀리 떨어진 데이터 포인트 간의 연결을 학습하게 한다. 평창 합성곱의 $L$개 계층 $l \in \{1, \dots, L\}$의 구조에 대해 dilated convolution은 다음과 같이 적층으로 쌓인 'feature map'을 출력한다.

$$\left[ K^{(f)} *_{d(l)} f^{(l-1)}\right]_i = \sum_{p=-k}^k K_p^{(l)} f_{d^{(l)}(i-1)+1+p}^{(l-1)}, i \in \left\{ 1, \dots, \lceil \frac{m}{d^{(l)}} \rceil\right\}$$

여기서 $d$는 dilation factor이고, 2의 배수씩 증가하도록 팽창을 선택할 수 있다. 즉, $d^{(l)} = 2^{l-1}$이다. 각 층에 대한 필터 $K^{(l)}$은 크기 $1 \times (2k + 1) = 1 \times 2$로 선택된다. 정규 합성곱 대신 팽창 합성곱을 사용하면 출력 $y$가 입력의 더 많은 노드에 의해 영향을 받을 수 있다. 신경망의 입력은 시계열 $X$에 의해 주어진다. 각 후속 층에서 팽창 합성곱을 적용하면 비선형성과 출력 feature map $f^{(l)}, l \in \{1, \dots, L\}$을 얻는다. 시계열의 다음 값을 예측하는 데 관심이 있으므로 모델을 훈련해서 출력이 예측된 시계열 $\hat{Y} = \{\hat{y}_t\}_{t=1}^N$이 되게 한다.

뉴런의 수용 영역은 그 뉴런의 출력값을 수정하는 입력의 원소 집합으로 정의됐었다. 이제 모델의 수용 영역이 $r$이 마지막 계층의 출력을 수정할 수 있는 첫째 계층의 입력 뉴런 수가 되도록 정의하자. 그러면 이는 계층 수 $L$과 필터 크기 $2k+1$에 의존하며 다음과 같이 주어진다.

$$r := 2^{L-1}(2k+1)$$